In [ ]:
from keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)


In [ ]:
import keras
from keras.models import Model    
from keras.layers import *
from keras.models import Sequential
from keras.layers.core import Reshape, Activation, Dropout
from keras.layers import LSTM, Dense
from keras.utils.vis_utils import plot_model
def VQA_MODEL():
    

    image_feature_size          = 2048
    word_feature_size           = 300
    number_of_LSTM              = 3
    number_of_hidden_units_LSTM = 512
    max_length_questions        = 30
    number_of_dense_layers      = 3
    number_of_hidden_units      = 1024
    activation_function         = 'tanh'
    dropout_pct                 = 0.5


    # Image model
    model_image = Sequential()
    model_image.add(Reshape((image_feature_size,), input_shape=(image_feature_size,)))
    model_image.add(Dense(1024, activation=activation_function))
    
    
    #caption model
    

    # Language Model


    inputs1 = Input(shape=(max_length_questions, word_feature_size))
    lstm1,h,c = LSTM(number_of_hidden_units_LSTM, return_sequences=True,return_state=True)(inputs1)
    lstm1,h,c = LSTM(number_of_hidden_units_LSTM, return_sequences=True,return_state=True)([lstm1,h,c])
    lstm1,h,c = LSTM(number_of_hidden_units_LSTM, return_sequences=True,return_state=True)([lstm1,h,c])

    lstm1,h,c = LSTM(number_of_hidden_units_LSTM, return_sequences=True,return_state=True)([lstm1,h,c])

    lstm1,h,c = LSTM(number_of_hidden_units_LSTM, return_sequences=False,return_state=True)([lstm1,h,c])
    #y = Lambda(lambda x: tf.keras.backend.concatenate([h,c],0))([lstm1,h,c])
    y = Concatenate()([h,c])
    model_language = Model(inputs=inputs1, outputs=y)
    # combined model
    conc = keras.layers.Multiply()([model_language.output,model_image.output])


    out = Dense(number_of_hidden_units, activation=activation_function)(conc)
    #out = Dropout(0.4)(out)

    out = Dense(number_of_hidden_units, activation=activation_function)(out)
    out = Dropout(0.5)(out)

    out = Dense(number_of_hidden_units, activation=activation_function)(out)
    out = Dropout(0.7)(out)




    out = Dense(1000, activation='softmax')(out)

    model_vqa = Model([model_language.input, model_image.input], out)


    #model_vqa.summary()
    #plot_model(model_vqa, to_file='model_vqa.png')

    return model_vqa

In [ ]:
def get_VQA_model():
    
    vqa_model = VQA_MODEL()

    vqa_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top_3_accuracy])
    return vqa_model

In [ ]:
from keras.models import load_model
model_vqa = get_VQA_model()
model_vqa.summary()

In [ ]:
import numpy as np

a = np.load('imgfet_res.npy')
b = np.load('quesfet.npy')
l = np.load('label.npy')

In [ ]:
a_train = a[0:60000]
b_train = b[0:60000]
l_train = l[0:60000]

a_test = a[60000:]
b_test = b[60000:]
l_test = l[60000:]

In [ ]:
############################runn################
import keras
from time import time
#from keras.callbacks import TensorBoard

#l = np.asarray(l)
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

train_y = keras.utils.np_utils.to_categorical(l_train,num_classes=1000)
len(train_y[0])
history1 = model_vqa.fit([b_train, a_train],train_y,batch_size=1000,epochs=80)


In [ ]:
######################################################

import matplotlib.pyplot as plt
y=history.history['loss']
x=range(0,80)
plt.plot(x,y)
#plt.title('top 5 accuracy')
plt.xlabel('number of epochs')
plt.ylabel('loss')
plt.show
plt.savefig('loss_normal.png',dpi=300,transparent=False)
plt.show

In [ ]:
import keras
train_y = keras.utils.np_utils.to_categorical(l_test,num_classes=1000)
loss,test_acc,top5,top3 = model_vqa.evaluate([b_test,a_test], train_y)

print('Test accuracy:', top5)


In [ ]:

# serialize model to JSON
model_json = model_vqa.to_json()
with open("model_vqa_vgg19_5layers.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_vqa.save("model_vqa_vgg19_5layers.h5")
print("Saved model to disk")